In [ ]:
import os
import shutil
import subprocess

python = "python"
python_helper_path = "/home/jovyan/utils/integration/git_helper.py"
base_repo = os.environ.get("GIT_REPOSITORY_URL", "https://github.com/Netcracker/qubership-env-checker.git")
base_sparse_folder = "jovyan/tests/"
base_sparse_file = "jovyan/tests/notebooks/test_notebook.ipynb"
default_target = "/home/jovyan/test1"
real_side_branch = "broadcast-pr-auto-assigner"
default_branch = "main"


def remove_folder(folder_path: str):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Removed: {folder_path}")
    else:
        print(f"Not found: {folder_path}")


def clear_git_credential_helper():
    # remove automatic substitution of saved credentials
    subprocess.run(["git", "config", "--global", "--unset", "credential.helper"], check=False)


def print_run_info(stdout: str, stderr: str, exit_code: str):
    print("====================================")
    print(stdout)
    print(stderr)
    print(f"Exit code: {exit_code}")
    print("====================================")

# Running with non-existent credentials is not possible, as there is no way
# to substitute them at runtime (requires separate check via env variables)

In [ ]:
# Regular integration run. Including check to fetch base_sparse_file.
# ER: successful execution
clear_git_credential_helper()
result = subprocess.run([
    python,
    python_helper_path,
    base_repo,
    default_target,
    base_sparse_file,
    default_branch
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)

In [ ]:
# Check attempt to fetch non-existent Git repository.
# ER: Error
clear_git_credential_helper()

result = subprocess.run([
    python,
    python_helper_path,
    "https://github.com/Netcracker/not-exists-xyz.git",
    default_target,
    base_sparse_file,
    default_branch
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)

In [ ]:
# Regular file creation inside folder
# ER: file will be saved with nesting level = 1
clear_git_credential_helper()

result = subprocess.run([
    python,
    python_helper_path,
    base_repo,
    "/home/jovyan/tmp_single",
    base_sparse_file,
    default_branch
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)

In [ ]:
# Creating file with deep nesting
# ER: file will be saved with nesting level > 1;
deep_target = "/home/jovyan/tmp/deep/a/b/c"
clear_git_credential_helper()

result = subprocess.run([
    python,
    python_helper_path,
    base_repo,
    deep_target,
    base_sparse_file,
    default_branch
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)

expected_file = f"{deep_target}/test_notebook.ipynb"
print("\n=== Check Result ===")
print(f"Target path exists: {os.path.exists(deep_target)}")
print(f"Sparse file exists: {os.path.exists(expected_file)}")

if os.path.exists(expected_file):
    print(f"✅ SUCCESS: File {expected_file} was fetched to {deep_target}")
else:
    print(f"❌ FAILED: File {expected_file} was not found in {deep_target}")

    if os.path.exists(deep_target):
        print(f"Contents of {deep_target}: {os.listdir(deep_target)}")

In [ ]:
# Check possibility to fetch not only file, but entire folder
# ER: entire folder will be fetched (sparse_path)
clear_git_credential_helper()

result = subprocess.run([
    python,
    python_helper_path,
    base_repo,
    default_target,
    base_sparse_folder,
    default_branch
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)

In [ ]:
# Attempt to fetch non-existent file (sparse_path) from repository
# ER: Error
clear_git_credential_helper()

result = subprocess.run([
    python,
    python_helper_path,
    base_repo,
    default_target,
    "jovyan/notebooks/does_not_exist.ipynb",
    default_branch
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)

In [ ]:
# Attempt to fetch not from main branch (actually existing)
# ER: Successful execution
# Notice: May need to change real_side_branch value
clear_git_credential_helper()

result = subprocess.run([
    python,
    python_helper_path,
    base_repo,
    default_target,
    base_sparse_folder,
    real_side_branch
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)

In [ ]:
# Attempt to fetch from non-existent branch
# ER: error
clear_git_credential_helper()

result = subprocess.run([
    python,
    python_helper_path,
    base_repo,
    default_target,
    base_sparse_folder,
    "branch-does-not-exist"
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)

In [ ]:
# subfolder specified
# ER: following structure will be created <GIT_TARGET_PATH>/projA/subA/<GIT_FILE>
clear_git_credential_helper()

result = subprocess.run([
    python,
    python_helper_path,
    base_repo,
    default_target,
    base_sparse_file,
    default_branch,
    "projA/subA",
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)

In [ ]:
# subfolder not specified
# ER: following structure will be created <GIT_TARGET_PATH>/empty/<GIT_FILE>
clear_git_credential_helper()

result = subprocess.run([
    python,
    python_helper_path,
    base_repo,
    default_target,
    base_sparse_folder,
    default_branch,
    "",
], capture_output=True, text=True)

print_run_info(result.stdout, result.stderr, result.returncode)